In [ ]:
# !pip3 install igraph
# pip install pymde
# !pip install scanpy
# !conda install -c conda-forge leidenalg
#%pip install hyperopt "ray[tune]"
#!pip install -U ipywidgets

In [2]:
import scanpy as sc
import scvi
import anndata as ad
from scvi.model.utils import mde
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
from scvi.autotune import ModelTuner
from ray import tune
import ray
seed = 9999
np.random.seed(seed)

In [3]:
adata = sc.read("data/annotated_file/oncofetal/Sub-clustering of endothelial and myeloid cells from 14 HCC patients/HCC_endo.h5ad")

/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [9]:
adata.obs

,n_genes,percent_mito,n_counts,louvain,patient_id,patient_tumorsection,NormalvsTumor,patientno,PNC,PIC,ViralvsNonViral,mean genes exp,median genes exp
index,,,,,,,,,,,,,
AGCTTGACAAACTGTC-1,796,0.014875,1479.0,2,1,P1_N,N,P1,N,N,Positive,0.635611,0.693147
ATTGGTGCACCGATAT-1,984,0.016029,1934.0,5,1,P1_N,N,P1,N,N,Positive,0.277259,0.000000
CAACCAAAGAGTCGGT-1,814,0.034839,1550.0,2,1,P1_N,N,P1,N,N,Positive,0.818869,0.000000
CAGAGAGAGCACAGGT-1,465,0.036723,708.0,5,1,P1_N,N,P1,N,N,Positive,0.000000,0.000000
CAGCTAATCACAGGCC-1,1307,0.016626,3248.0,2,1,P1_N,N,P1,N,N,Positive,0.693147,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCTCCTAAGTACCGGA-57,453,0.021705,645.0,1,57,P15_T3,T,P15,C,C,Negative,0.277259,0.000000
TCTTTCCGTTTAAGCC-57,222,0.020649,339.0,1,57,P15_T3,T,P15,C,C,Negative,0.000000,0.000000
CAGCTAACATTATCTC-58,2008,0.018675,5569.0,3,58,P15_T4T5,T,P15,P,P,Negative,1.075056,1.242453


In [2]:
#adata = sc.read("h5ad_data/RasV.h5ad") # our doublet removed individualy sct dataset
#adata = sc.read('SCT/SCVI_integration/res_0.6_tuned/RasV_ingr_scvi_tuned_06.h5ad') # our scvi hypert tuned annotated dataset

In [3]:
adata.obs = adata.obs.loc[:, ~adata.obs.columns.str.startswith('DF')]
## here i just had some extra metadata column that idont need aymroe

In [4]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,condition,replicate,percent.mt,nCount_SCT,nFeature_SCT,SCT_snn_res.0.08,seurat_clusters,SCT_snn_res.0.07,batch
ctl_1_AAACCCAAGAAGTCTA-1,NASH_HCC_RASV,6545.0,2666,ctl,1,6.661574,6488.0,2658,1,1,1,ctl_1
ctl_1_AAACCCAAGAGCAGTC-1,NASH_HCC_RASV,7690.0,2660,ctl,1,6.176853,6819.0,2644,2,2,2,ctl_1
ctl_1_AAACCCAAGCCTCATA-1,NASH_HCC_RASV,16496.0,4806,ctl,1,2.764306,6135.0,2748,3,3,3,ctl_1
ctl_1_AAACCCAAGGTCATCT-1,NASH_HCC_RASV,9218.0,3493,ctl,1,3.070080,6960.0,3484,1,1,1,ctl_1
ctl_1_AAACCCAAGGTCGACA-1,NASH_HCC_RASV,8313.0,2788,ctl,1,3.969686,6917.0,2769,3,3,3,ctl_1
...,...,...,...,...,...,...,...,...,...,...,...,...
end_3_TTTGTTGCATCGATGT-1,NASH_HCC_RASV,1014.0,503,end,3,7.790927,5386.0,1285,4,2,2,end_3
end_3_TTTGTTGTCGACCATA-1,NASH_HCC_RASV,19197.0,4797,end,3,1.604417,6610.0,2674,2,4,4,end_3
end_3_TTTGTTGTCGAGTCTA-1,NASH_HCC_RASV,5713.0,2004,end,3,3.010677,6161.0,1990,7,7,7,end_3
end_3_TTTGTTGTCGCAGTCG-1,NASH_HCC_RASV,5445.0,2612,end,3,11.533517,5941.0,2598,1,1,1,end_3


In [5]:
adata.layers["counts"] = adata.X.copy()

In [6]:
adata.X.shape

(73268, 24461)

## Normalize the data ( not for inputing them on ScVI model though!)
highly variable genes can be inputed in the model

In [7]:
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
# adata.raw= adata

In [8]:
#sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True, layer = "counts", flavor= 'seurat_v3', batch_key = "batch")

## Model setup and train! skip if i already trained it

# 1. With Hyper parameter tuning

In [9]:
model = scvi.model.SCVI

In [10]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="condition", continuous_covariate_keys =  ["percent.mt","nCount_RNA"])

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [11]:
tuner = scvi.autotune.ModelTuner(model)

In [12]:
tuner.info()

ModelTuner registry for SCVI

                  Tunable hyperparameters                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃      Hyperparameter      ┃ Default value ┃    Source    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│         n_hidden         │      128      │     VAE      │
│         n_latent         │      10       │     VAE      │
│         n_layers         │       1       │     VAE      │
│       dropout_rate       │      0.1      │     VAE      │
│        dispersion        │     gene      │     VAE      │
│     log_variational      │     True      │     VAE      │
│     gene_likelihood      │     zinb      │     VAE      │
│   latent_distribution    │    normal     │     VAE      │
│    encode_covariates     │     False     │     VAE      │
│ deeply_inject_covariates │     True      │     VAE      │
│      use_batch_norm      │     both      │     VAE      │
│      use_layer_norm      │     none      │     VAE      │
│  use_observed_lib_size   │     True      │     VAE      │
│      var_activation      │     None      │     VAE      │
│        optimizer         │     Adam      │ TrainingPlan │
│            lr            │     0.001     │ TrainingPlan │
│       weight_decay       │     1e-06     │ TrainingPlan │
│           eps            │     0.01      │ TrainingPlan │
│    n_steps_kl_warmup     │     None      │ TrainingPlan │
│    n_epochs_kl_warmup    │      400      │ TrainingPlan │
│   reduce_lr_on_plateau   │     False     │ TrainingPlan │
│        lr_factor         │      0.6      │ TrainingPlan │
│       lr_patience        │      30       │ TrainingPlan │
│       lr_threshold       │      0.0      │ TrainingPlan │
│          lr_min          │       0       │ TrainingPlan │
│      max_kl_weight       │      1.0      │ TrainingPlan │
│      min_kl_weight       │      0.0      │ TrainingPlan │
│        batch_size        │      128      │     SCVI     │
└──────────────────────────┴───────────────┴──────────────┘

       Available metrics        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃     Metric      ┃    Mode    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ validation_loss │    min     │
└─────────────────┴────────────┘

                         Default search space                         
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Hyperparameter ┃ Sample function ┃  Arguments  ┃ Keyword arguments ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│    n_hidden    │     choice      │ [[64, 128]] │        {}         │
└────────────────┴─────────────────┴─────────────┴───────────────────┘

In [13]:
ray.init(log_to_driver=False)
search_space ={"n_hidden": tune.choice([92, 128, 192, 256]),
    "n_latent": tune.choice([10, 20, 30, 40, 50, 60]),
    "n_layers": tune.choice([1, 2, 3]),
    "lr": tune.choice([1e-4, 1e-2]),
    "gene_likelihood": tune.choice(["nb", "zinb"])}


/home/crow/anaconda3/envs/singlecell/lib/python3.9/subprocess.py:1770: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-05-03 00:49:23,166	INFO worker.py:1724 -- Started a local Ray instance.


(raylet) Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 1807, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1908, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1813, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1754, in ray._raylet.execute_task.function_executor
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/_private/function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/util/tracing/tracing_helper.py", line 467, in _resume_span
    return method(self, *_args, **_kwargs)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 342, in train
    raise skipped from exception_cause(skipped)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/air/_internal/util.py", line 

In [14]:
results = tuner.fit(adata, metric = "validation_loss",
                   resources = {"gpu":1, "cpu":16}, #use gpu
                   search_space = search_space, # parameter grid
                   num_samples = 100, # combinations of hyperparams
                   max_epochs = 20) # epochs for each combination


2024-05-03 00:49:28,953	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-05-03 01:40:05,280	ERROR tune_controller.py:1374 -- Trial task failed for trial _trainable_3ef46797
Traceback (most recent call last):
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/ray/_private/worker.py", line 262

In [16]:
results.results.get_best_result
results.metric

{'metric': 'validation_loss', 'mode': 'min', 'value': 8265.39453125}

## best validation loss (lower)
  Result(
    metrics={'validation_loss': 8230.080078125},
    path='/mnt/e/NKI_ScAnalysis/Analysis/scvi_log/autotune/2024-05-02_18-02-35_scvi/_trainable_cda30ce3_87_gene_likelihood=zinb,lr=0.0100,n_hidden=192,n_latent=60,n_layers=1_2024-05-02_19-48-09',
    filesystem='local',
    checkpoint=None
  ),

# 2 With standard model

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="batch", continuous_covariate_keys =  ["percent.mt","nCount_RNA"])

In [ ]:
# defaults
model = scvi.model.SCVI(adata, n_layers=2, n_latent=30, gene_likelihood="nb")

# With tuning

In [ ]:
# with hyper param tuning
model = scvi.model.SCVI(adata,gene_likelihood="zinb",n_hidden=192,n_latent=60,n_layers=1)
kwargs ={'lr':0.0100}

In [ ]:
model.train(max_epochs=200, early_stopping = True, plan_kwargs=kwargs)
model.save('all_genes_tuned_modelv3')

## Load already trained models

In [17]:
#model.save(dir_path = "models/allgenesv3", overwrite=True)
model = scvi.model.SCVI.load(dir_path="models/allgenesv3", adata=adata)
#model = scvi.model.SCVI.load(dir_path="SCT/SCVI_integration/res_0.6_tuned/all_genes_tuned_model", adata=adata)

INFO     File models/allgenesv3/model.pt already downloaded                                                        


In [18]:
latent = model.get_latent_representation()
latent.shape

(73268, 60)

In [19]:
adata.obsm["X_scvi"] = latent
adata.layers["scvi_normalized"] = model.get_normalized_expression(library_size = 1e4)

In [20]:
#adata.layers["scvi_normalized"]

In [21]:
sc.pp.neighbors(adata, use_rep = "X_scvi",)
sc.tl.umap(adata, random_state=seed)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'print':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "../../../../home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/pynndescent/pynndescent_.py", line 253:[0m
[1mdef nn_descent_internal_low_memory_parallel(
    <source elided>
        if verbose:
[1m            print("\t", n + 1, " / ", n_iters)
[0m            [1m^[0m[0m
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function nn_descent_internal_low_memory_parallel at 0x7f94ac77d310>))[0m
[0m[1mDuring: typing of call at /home/crow/anaconda3/envs/singlecell/lib/python3.9/site-packages/pynndescent/pynndescent_.py (359)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function nn_descent_internal_low_memory_parallel at 0x7f94ac77d310>))[0m
[0m[1mDuring: typing of call at /home/crow

## Clustering based on the leiden alg

In [ ]:
sc.tl.leiden(adata, resolution = 0.1, random_state=seed, flavor= "igraph", n_iterations=2, directed = False)#, n_iterations=2, directed= False

## MDE(autoencoders) OR Classic pca umap for visualization!

In [ ]:
adata.obsm["X_mde"] = mde(adata.obsm["X_scvi"])

In [ ]:
sc.pl.embedding(
    adata,
    basis="X_mde",
    color=["condition", "leiden"],
    frameon=False,
    ncols=1,
    save='mde_cell_types0_06_allgenes.pdf'
)

In [ ]:
sc.pl.umap(adata, color = ["leiden"], frameon = False , save= "umap_cell_type.pdf")
sc.pl.umap(adata, color = ["condition"], frameon = False , save= "umap_condition.pdf")

## Markers based on DEG for each cluster against the others

In [ ]:
df = model.differential_expression(groupby = 'leiden')
markers = {}
for c in adata.obs.leiden.cat.categories:
    cell_df = df.loc[df.group1 == c]
    markers[c] = cell_df.index.tolist()[:100]

In [ ]:
import pandas as pd


markers_df = pd.DataFrame.from_dict(markers, orient='index').transpose()

markers_df.columns = [f"Cluster_{col}" for col in markers_df.columns]
markers_df.index = [f"Marker_{i+1}" for i in markers_df.index]
print(markers_df.head())


In [ ]:
#!pip install openpyxl
markers_df.to_excel('markers_1_scvi_allgenes_tuned_v3.xlsx', index=True)


In [ ]:
#adata.write('RasV_ingr_scvi.h5ad')

In [ ]:
#adata = sc.read('your_filename.h5ad')

## Manual Annotation

In [ ]:
cluster_cell_types = { ### 0.6
    "0": 'EC',         # Endothelial cells, possibly liver sinusoidal endothelial cells
    "1": 'EC',         # Vascular endothelial cells and hepatic stellate cells
    '2': 'Bcells',     # B cells
    '3': 'T',          # T cells
    '4': 'DC',         # Dendritic cells, but potentially mixed immune cells, "T" may also be considered
    '5': 'Myeloid',    # Myeloid cells, predominantly macrophages
    '6': 'Hep1/cc',    # Hepatocytes (Hep1/cc seems appropriate)
    '7': 'Hep/Cholangi',  # Cholangiocytes or bile duct cells
    '8': 'Gran',       # Neutrophils and other granulocytes
    '9': 'plasma/actB'}

Cluster_1: EC - Endothelial Cells
Cluster_3: T Cells - Diverse T Cell Population
Cluster_4: Hep - Hepatocytes
Cluster_6: Fib/Mes - Fibroblasts/Mesenchymal Cells
Cluster_7: End/Peri - Endothelial Cells/Pericytes
Cluster_8: Epith - Epithelial Cells
Cluster_9: DC - Dendritic Cells and Immune Cells in Mucosal Sites
Cluster_10: T Cells - Regulatory and Activated T Cells
Cluster_11: Eryth - Erythroid Cells
Cluster_12: Neut - Neutrophils in Inflammatory Response
Cluster_13: B Cells - Mature B Cells in Germinal Cente

In [ ]:
cluster_cell_types = { ### 0.6
    "0": 'EC',         # Endothelial cells, possibly liver sinusoidal endothelial cells
    "1": 'EC',         # Vascular endothelial cells and hepatic stellate cells
    '2': 'B',     # B cells
    '3': 'T',          # T cells
    '4': 'HEP',         # Dendritic cells, but potentially mixed immune cells, "T" may also be considered
    '5': 'Myeloid',    # Myeloid cells, predominantly macrophages
    '6': 'Fib/Mes',    #
    '7': 'EC',  # Cholangiocytes or bile duct cells
    '8': 'HEP/epith',       # Neutrophils and other granulocytes
    '9': 'plasma/actB'
    '10': 'T'
    '11': 'ery'
    '12': 'neut'
    '13': 'B'
}

In [ ]:
adata.obs['cell_types'] = ''

# the new column with the corresponding cell types based on the leiden clustering numbers
for cluster_num, cell_type in cluster_cell_types.items():
    adata.obs.loc[adata.obs['leiden'] == cluster_num, 'cell_types'] = cell_type
print(adata.obs.head())

In [ ]:
sc.pl.umap(adata, color=["cell_types", "condition"], frameon=False, save="annotated_scvi.png")


In [ ]:
#The continous creates issue when transformed to seurat object in R so we could delete it
#del adata.obsm['_scvi_extra_continuous_covs']
adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
adata.write('RasV_ingr_scvi_tuned_06.h5ad')